In [ ]:
from __future__ import print_function
from __future__ import division

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
import torchvision
from torchvision import transforms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import time
import os
import copy
from tqdm.notebook import tqdm

In [ ]:
os.chdir('../')

In [ ]:
from input.yelpimports.lightning_model import init_cls_model
from input.yelpimports.dataset_loaders import build_datasets, YelpDataset
from input.yelpimports.metrics import multiclass_stats

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print("PyTorch Lightning Version: ",pl.__version__)

In [ ]:
!nvidia-smi -L

In [ ]:
FLAGS = {
    'model_name': 'custom',
    'photo_dir': 'input/yelpbusinessrestaurant/br_photos/photos/',
    'csv_path': 'input/yelpbusinessrestaurant/business_restaurant.csv',
    'output_dim': 2, #6,
    'batch_size': 512, # 512 / N gpu
    'num_workers': 4,  # 4 per gpu
    'learning_rate': 0.02,  # 0.02 * N gpu
    'max_epochs': 20,  # arbitrary
    'feature_extract': True,
    'use_pretrained': True,
    'multilabel': False,
    'threshold': 0.5,
    'class_weight': [1,1]
        }

In [ ]:
csv_file = pd.read_csv(FLAGS['csv_path'])

In [ ]:
assert len(os.listdir(FLAGS['photo_dir'])) == len(csv_file)

In [ ]:
yelp_model, input_size = init_cls_model(FLAGS)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
print("Initializing Datasets and Dataloaders...")

In [ ]:
csv_file.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(csv_file, test_size=0.2, random_state=42)
dev, test = train_test_split(test, test_size=0.5, random_state=42)

In [ ]:
data_frames = {'train': train,
              'dev': dev,
              'test': test}

In [ ]:
dataloaders_dict = build_datasets(FLAGS, data_frames, data_transforms)

In [ ]:
dataloaders_dict

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
trainer = pl.Trainer(callbacks=[checkpoint_callback, early_stop_callback],
                     progress_bar_refresh_rate=1, max_epochs=FLAGS['max_epochs'],
                     stochastic_weight_avg=True, precision=16, gpus=-1)

In [ ]:
trainer.fit(yelp_model, dataloaders_dict['train'], dataloaders_dict['dev'])

In [ ]:
if 'checkpoints' not in os.listdir('working'):
    os.mkdir('working/checkpoints')
trainer.save_checkpoint(f"working/checkpoints/{FLAGS['model_name']}_amb.ckpt")

In [ ]:
print(checkpoint_callback.best_model_path)

In [ ]:
yelp_model, _ = init_cls_model(FLAGS, checkpoint_callback.best_model_path)

In [ ]:
trainer.test(yelp_model, dataloaders_dict['test'])

In [ ]:
model = yelp_model.eval().cuda(device=0)

In [ ]:
y_pred, y_true = [], []

In [ ]:
test_dataset = YelpDataset(test, FLAGS['photo_dir'], data_transforms['test'])

In [ ]:
for image, labels in tqdm(test_dataset):
    output = model(image.unsqueeze(0).cuda(0)).cpu().data.numpy()[0]
    y_pred.append(output)
    y_true.append(labels.numpy().astype('int'))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
f1_score(y_true, y_pred)

In [ ]:
os.listdir('working/checkpoints')

In [ ]:
from IPython.display import FileLink
os.chdir('working')
FileLink(r"checkpoints/custom_amb.ckpt")